How many entries were made per day?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Make a dataframe showing only whether meals were successfully recorded, and remove nearby ones

"""
import pandas as pd
from itertools import islice

from ema import clean

# Read data in
meal_df = clean.cleaned_smartwatch(keep_catchups=False)

# Collapse meal type to positive/negative response
assert set(meal_df["meal_type"].unique()) == {
    "Meal",
    "Drink",
    "Snack",
    "No food/drink",
    "No response",
}, set(meal_df["meal_type"].unique())
meal_df["meal_type"] = meal_df["meal_type"].isin({"Meal", "Drink", "Snack", "No Food"})

meal_df["keep"] = True

prev_row = None
n_rows = len(meal_df)
# Collapse nearby entries to the same entry
# Could probably use itertools.pairwise here
for _, group in meal_df.groupby("p_id"):
    for (prev_time, prev_row), (next_time, next_row) in zip(
        islice(group.iterrows(), n_rows - 1), islice(group.iterrows(), 1, n_rows)
    ):
        # If this is the within 2min of the previous entry and they are both True, mark the later ones to remove
        if (
            next_time - prev_time < pd.Timedelta(minutes=2)
            and prev_row["meal_type"]
            and next_row["meal_type"]
        ):
            meal_df.loc[next_time, "keep"] = False

meal_df = meal_df[meal_df["keep"]]
meal_df.drop(columns=["keep"], inplace=True)

/home/mh19137/seaco_malaysia/ema/clean.py:189: UserWarning: Long catchup: not marking Meal at 2022-05-19 09:02:49 as catchup
  warnings.warn(


 Open ended catchup at 2022-02-11 20:02:05 ended by No response at 2022-02-12 08:02:11
 Open ended catchup at 2022-02-14 20:26:30 ended by No response at 2022-02-15 08:02:16
 Open ended catchup at 2022-04-19 08:01:14 ended by No catch-up at 2022-04-19 08:01:17
 Open ended catchup at 2022-06-27 15:53:14 ended by Catch-up start at 2022-06-27 15:54:06
 Open ended catchup at 2021-12-31 22:11:10 ended by No catch-up at 2022-01-01 08:01:15
 Open ended catchup at 2022-05-24 07:25:36 ended by Catch-up start at 2022-05-24 07:26:00
 Open ended catchup at 2022-03-21 08:01:57 ended by No catch-up at 2022-03-21 08:02:04
 Open ended catchup at 2022-03-07 18:33:30 ended by No catch-up at 2022-03-07 18:33:34
 Open ended catchup at 2022-03-07 18:37:08 ended by No catch-up at 2022-03-07 18:37:14
 Open ended catchup at 2022-03-07 19:29:23 ended by No response at 2022-03-08 08:02:56
 Open ended catchup at 2022-03-08 08:30:40 ended by No catch-up at 2022-03-08 08:31:08
 Open ended catchup at 2022-03-08 09: